In [0]:
#########################################################################
#########------- Introducción al Deep Learning ------------###############
#########################################################################
# Capacitador: André Omar Chávez Panduro
# email: andre.chavez@urp.edu.pe / andrecp38@gmail.com
# Tema : Redes Neuronales Artificiales
# version: 1.0
#########################################################################

In [1]:
# Nos vinculamos a nuestro drive, para poder usar facilmente GColab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Parte 1 - Pre procesado de datos
# Cómo importar las librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
# Importar el data set
data = pd.read_csv('/content/drive/My Drive/IntroduccionalDeepLearning/Churn_Modelling.csv')

In [5]:
# Revisamos los primeros registros del set de datos
data.head(5)
#data.shape

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
# Separamos el conjunto de features y target
# Esta es la matriz de caracteristicas, debo elegir adecuadamente las variables.
X = data.iloc[:, 3:13].values  # Features, Covariables, Drivers, VI.
y = data.iloc[:, 13].values    # Target.

In [7]:
# Revisamos los datos obtenidos!
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [0]:
# Tal cual lo estudiado , procedemos al LabelEncoder o OneHotEncoding
from sklearn.preprocessing import LabelEncoder # Importan o extraemos
labelencoder_X_1 = LabelEncoder()              # Define o instanciamos
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1]) # Lo hacemos para la variable geografia o pais.
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2]) # Le hago label encoder al genero

In [10]:
# Revisamos los datos obtenidos!
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [0]:
# Le hacemos OneHotEncoding a la variable pais
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(
    transformers=[
        ("Churn_Modelling",                # Un nombre de la transformación
         OneHotEncoder(categories='auto'), # La clase a la que transformar
         [1]                               # Las columnas a transformar.
         )
    ], remainder='passthrough'
)


In [0]:
# Le aplicamos la transformacion y nos quedamos con las nuevas categorias -1.
X = transformer.fit_transform(X)
X = X[:, 1:]


In [0]:
# Dividir el data set en conjunto de entrenamiento y conjunto de testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.3, 
                                                    stratify=y,
                                                    random_state = 0)

In [0]:
# Importante !!!
# Escalado de variables
from sklearn.preprocessing import StandardScaler # Importan el modulo
sc_X = StandardScaler()                          # Definir la funcion
X_train = sc_X.fit_transform(X_train)            # Ajuste
X_test = sc_X.transform(X_test)                  # Aplicacion

In [15]:
X_train

array([[ 1.73734559, -0.5740511 , -2.19308136, ...,  0.64620273,
        -1.03904751, -0.84136356],
       [ 1.73734559, -0.5740511 ,  2.07600997, ..., -1.54750198,
         0.9624199 ,  1.23668977],
       [ 1.73734559, -0.5740511 , -0.3175825 , ..., -1.54750198,
        -1.03904751, -1.32226278],
       ...,
       [-0.57559072, -0.5740511 , -0.40047748, ..., -1.54750198,
         0.9624199 , -0.58628398],
       [-0.57559072, -0.5740511 ,  0.74969033, ...,  0.64620273,
         0.9624199 , -1.47588406],
       [-0.57559072,  1.7420052 ,  0.96728965, ...,  0.64620273,
        -1.03904751,  0.51770707]])

In [0]:
# Parte 2 - Construir la RNA

In [0]:
# Importar Keras y librerías adicionales
import keras
from keras.models import Sequential # Arquitectura de la red
from keras.layers import Dense      # Capas de las redes neuronales
from keras.layers import Dropout    # Regularizacion / Evita sobreajuste

In [20]:
# Inicializar la RNA.
# Si la definimos como secuencia de capas es mucho mas facil despues trabajar con ellas.
classifier = Sequential() # Podria codificar aca las capas y demas pero lo hacemos una a una.

RuntimeError: ignored

In [0]:
# Añadir las capas de entrada y primera capa oculta
# Dense es un tipo de capa, la necesitamos codificar. 
# units : Numero de neuronas
# kernel_initializer : Inicializacion de pesos.
# activation : Funcion de activacion
# input_dim : Es el numero de variables o X o features.
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu", input_dim = 11)) # Primera capa oculta
#classifier.add(Dropout(p = 0.1))

In [0]:
# Añadir la segunda capa oculta
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))
#classifier.add(Dropout(p = 0.1)) # Regularización

In [0]:
# Añadir la capa de salida
# Solo necesitamos una unidad pues tenemos un resultado binario.
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))
# Si tengo mas de 2 categorias , puedo mantener la funcion de activacion sigmoide?

In [0]:
# Compilar la RNA
# Podemos probar con distintos optimizadores
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Ajustamos la RNA al Conjunto de Entrenamiento
# batch_size : Numero de observaciones antes de actualizar los pesos/ tamaño del bloque a procesar.
# epochs     : Numero de iteracciones globales / Mas veces mas aprendizaje u overfitting.
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)





Epoch 1/100





7000/7000 [==============================] - 2s 216us/step - loss: 0.4950 - acc: 0.7951
Epoch 2/100
7000/7000 [==============================] - 1s 115us/step - loss: 0.4292 - acc: 0.7963
Epoch 3/100
7000/7000 [==============================] - 1s 123us/step - loss: 0.4260 - acc: 0.7963
Epoch 4/100
7000/7000 [==============================] - 1s 121us/step - loss: 0.4235 - acc: 0.7963
Epoch 5/100
7000/7000 [==============================] - 1s 120us/step - loss: 0.4198 - acc: 0.8114
Epoch 6/100
7000/7000 [==============================] - 1s 117us/step - loss: 0.4162 - acc: 0.8219
Epoch 7/100
7000/7000 [==============================] - 1s 119us/step - loss: 0.4136 - acc: 0.8281
Epoch 8/100
7000/7000 [==============================] - 1s 116us/step - loss: 0.4120 - acc: 0.8326
Epoch 9/100
7000/7000 [==============================] - 1s 121us/step - loss: 0.4104 - acc: 0.8311
Epoch 10/100
7000/7000 [==============================] - 1s 117us/step - loss: 0.4091 - acc

In [0]:
# Parte 3 - Evaluar el modelo y calcular predicciones finales

# Predicción de los resultados con el Conjunto de Testing
y_pred_train  = classifier.predict(X_train)
y_pred_train = (y_pred_train>0.5)
y_pred_test  = classifier.predict(X_test)
y_pred_test = (y_pred_test>0.5)


In [0]:
# Elaborar una matriz de confusión
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score
cm_train = confusion_matrix(y_train, y_pred_train)
cm_test = confusion_matrix(y_test, y_pred_test)

accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)


print(cm_train)
print(cm_test)
print(accuracy_train)
print(accuracy_test)

[[5429  145]
 [ 983  443]]
[[2328   61]
 [ 435  176]]
0.8388571428571429
0.8346666666666667


In [0]:
# Tarea: Hallar y comparar los siguientes indicadores: Sensibilidad, Especificidad, Precision y F1-Score

In [0]:
## Parte 4 - Evaluar, mejorar y Ajustar la RNA

In [0]:
### Evaluar la **RNA**
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [0]:
# Construimos un clasificador

In [0]:
def build_classifier():
  classifier = Sequential()
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 11))
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu"))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))
  classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier

In [0]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, nb_epoch = 100)
accuracies = cross_val_score(estimator=classifier, X = X_train, y = y_train, cv = 10, n_jobs=-1, verbose = 1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   13.8s finished


In [0]:
# Corroboramos si estamos ante un problema de Bias o Variance
mean = accuracies.mean()
variance = accuracies.std()

In [0]:
print(mean)
print(variance)

0.7962857111011232
0.016941675700151964


In [0]:
### Mejorar la RNA
#### Regularización de Dropout para evitar el *overfitting*

In [0]:
### Ajustar la *RNA*
from sklearn.model_selection import GridSearchCV # sklearn.grid_search

In [0]:
def build_classifier(optimizer):
  classifier = Sequential()
  classifier.add(Dense(units = 16, kernel_initializer = "uniform",  activation = "relu", input_dim = 11))
  classifier.add(Dropout(p = 0.1))
  classifier.add(Dense(units = 16, kernel_initializer = "uniform",  activation = "relu"))
  classifier.add(Dropout(p = 0.1))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))
  classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier

In [0]:
# Construimos el clasificador
classifier = KerasClassifier(build_fn = build_classifier)

In [0]:
# Hacemos el grid con los parámetros que queremos optimizar
parameters = {
    'batch_size' : [25,32],
    'nb_epoch' : [100, 250 , 500], 
    'optimizer' : ['adam', 'rmsprop']
}

In [0]:
grid_search = GridSearchCV(estimator = classifier, 
                           param_grid = parameters, 
                           scoring = 'accuracy', 
                           cv = 3)
grid_search = grid_search.fit(X_train, y_train)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  


Epoch 1/1
4666/4666 [==============================] - 1s 113us/step - loss: 0.5465 - acc: 0.7958
Epoch 1/1
4667/4667 [==============================] - 1s 140us/step - loss: 0.5513 - acc: 0.8012
Epoch 1/1
4667/4667 [==============================] - 1s 152us/step - loss: 0.5593 - acc: 0.7874
Epoch 1/1
4666/4666 [==============================] - 1s 161us/step - loss: 0.5438 - acc: 0.7951
Epoch 1/1
4667/4667 [==============================] - 1s 167us/step - loss: 0.5525 - acc: 0.7997
Epoch 1/1
4667/4667 [==============================] - 1s 171us/step - loss: 0.5485 - acc: 0.7898
Epoch 1/1
4666/4666 [==============================] - 1s 202us/step - loss: 0.5635 - acc: 0.7928
Epoch 1/1
4667/4667 [==============================] - 1s 217us/step - loss: 0.5462 - acc: 0.8007
Epoch 1/1
4667/4667 [==============================] - 1s 250us/step - loss: 0.5571 - acc: 0.7877
Epoch 1/1
4666/4666 [==============================] - 1s 233us/step - loss: 0.5647 - acc: 0.7953
Epoch 1/1
4667/4667 

In [0]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [0]:
print(best_parameters)
print(best_accuracy)

{'batch_size': 25, 'nb_epoch': 100, 'optimizer': 'adam'}
0.7962858079982782


In [0]:
# FIN !!